In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
img=plt.imread('MLSquadBW.jpg')
imgplot = plt.imshow(img, cmap = "gray")
plt.show()

In [ ]:
class NoisyImages(object):
    def __init__(self, prop, varSigma):
        self.prop = prop
        self.varSigma = varSigma
        
    def addGaussianNoise(self, im):
        im = im/255
        N = int(np.round(np.prod(im.shape)*self.prop))
        index = np.unravel_index(np.random.permutation(np.prod(im.shape))[1:N],im.shape)
        e = self.varSigma*np.random.randn(np.prod(im.shape)).reshape(im.shape)
        im2 = np.copy(im).astype('float')
        im2[index] += e[index]
        return im2
    
    def addSaltnPepperNoise(self, im):
        im = im/255
        N = int(np.round(np.prod(im.shape)*self.prop))
        index = np.unravel_index(np.random.permutation(np.prod(im.shape))[1:N],im.shape)
        im2 = np.copy(im)
        im2[index] = 1-im2[index]
        return im2
    

In [ ]:
createNoise = NoisyImages(0.7, 0.1)

gaussianImg = createNoise.addGaussianNoise(img)
snpImg = createNoise.addSaltnPepperNoise(img)

#rounding values to 1's and 0's
gaussianImg = gaussianImg.round()
print(gaussianImg)
snpImg = snpImg.round()
print(snpImg)

plt.imshow(gaussianImg, cmap="gray")
plt.title("Gaussian Noise")
plt.show()

plt.imshow(snpImg, cmap="gray")
plt.title("Salt and Pepper Noise")
plt.show()

In [ ]:
class HelperTools(object):    
    def neighbours(self, i, j, M, N, size=4):
        # M = size of rows (ie x), N = size of columns (ie y)
        if size == 4:
            if i == 0 and j == 0:
                n = [(0, 1), (1, 0)]
            elif i == 0 and j==N-1:
                n = [(0, N-2), (1, N-1)]
            elif i == M-1 and j == 0:
                n = [(M-1, 1), (M-2, 0)]
            elif i == M-1 and j == N-1:
                n = [(M-1, N-2), (M-2, N-1)]
            elif i == 0:
                n = [(0, j-1), (0, j+1), (1, j)]
            elif i == M-1:
                n = [(M-1, j-1), (M-1, j+1), (M-2, j)]
            elif j == 0:
                n = [(i-1, 0), (i+1, 0), (i, 1)]
            elif j == N-1:
                n = [(i-1, N-1), (i+1, N-1), (i, N-2)]
            else:
                n = [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]
                
        if size == 8:
            if i == 0 and j == 0:
                n = [(0, 1), (1, 0), (1, 1)]
            elif i == 0 and j == N-1:
                n = [(0, N-2), (1, N-1), (1, N-2)]
            elif i == M-1 and j == 0:
                n = [(M-1, 1), (M-2, 0), (M-2, 1)]
            elif i == M-1 and j == N-1:
                n = [(M-1, N-2), (M-2, N-1), (M-2, N-2)]
            elif i == 0:
                n = [(0, j-1), (0, j+1), (1, j), (1, j+1), (1, j-1)]
            elif i == M-1:
                n = [(M-1, j-1), (M-1, j+1), (M-2, j), (M-2, j+1), (M-2, j-1)]
            elif j == 0:
                n = [(i-1, 0), (i+1, 0), (i, 1), (i+1, 1), (i-1, 1)]
            elif j == N-1:
                n = [(i-1, N-1), (i+1, N-1), (i, N-2), (i+1, N-2), (i-1, N-2)]
            else:
                n = [(i-1, j), (i+1, j), (i, j-1), (i, j+1), (i-1, j-1), (i-1, j+1), (i+1, j-1), (i+1, j+1)]
            
        return n

In [ ]:
class ICM(object):
    def __init__(self, image):
        self.image = image
        self.T = image.shape[0]
        self.N = image.shape[1]
        self.helper = HelperTools()
        self.x = np.random.rand(self.T, self.N)
        self.x = self.x.round()
        for i in range(self.T):
            for j in range(self.N):
                if self.x[i][j] == 0:
                    self.x[i][j] = -1
    
    def e0(self, t, i):
        ni = self.helper.neighbours(t, i, self.T, self.N, 8)
        w = 1
        out = 0
        for j in ni:
            out += w*(self.x[t][i])*(self.x[j[0]][j[1]])
        return out
    
    def lixi(self, t, i):
        if self.x[t][i] == 1 and self.image[t][i] == 1:
            return 100
        elif self.x[t][i] == -1 and self.image[t][i] == 0:
            return 100
        elif self.x[t][i] == 1 and self.image[t][i] == 0:
            return 1
        elif self.x[t][i] == -1 and self.image[t][i] == 1:
            return 1
        
    def prodfunc(self, Z0, Z1, x, y, xiVal, pxi):
        prodOutput = 1
        initx = self.x[x][y]
        for i in range(self.T):
            for j in range(self.N):
                if not (i == x and j == y):
                    prodOutput *= np.exp(self.lixi(i, j))*(1/Z0)*np.exp(self.e0(i, j))
        self.x[x][y] = xiVal
        prodOutput *= np.exp(self.lixi(x, y))*(1/Z0)*np.exp(self.e0(x, y))*pxi
        prodOutput = (1/Z1)* prodOutput
        self.x[x][y] = initx
        
        return prodOutput

        
    def runICMloop(self):
        Z0 = 1
        Z1 = 1
        for t in range(self.T):
            for i in range(self.N):
                if self.prodfunc(Z0, Z1, t, i, 1, 0.5) >  self.prodfunc(Z0, Z1, t, i, -1, 0.5):
                    output = 1
                else:
                    output = -1
        return output
    
                
                

In [ ]:
ICMdoer = ICM(gaussianImg)

In [ ]:
output = ICMdoer.runICMloop()
print(output)